In [7]:
"""
Generating data for "Predicting Forest Regrowth in Amazonia given Land Use History"
Author: Ana Catarina Avila
Date: Dec 2023
"""

import ee
import geemap
from utils import import_roi

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

# specifies region around Paragominas for visualization and testing
plot_region = ee.Geometry.Rectangle([-47, -3.5, -46, -2.5])

roi = import_roi('br_amazon')

In [22]:
agbd = ee.Image("projects/ee-ana-zonia/assets/agbd")
age = ee.Image("projects/ee-ana-zonia/assets/age")

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoreg")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")

LU_sum = ee.Image("projects/ee-ana-zonia/assets/LU_sum")
last_fire = ee.Image("projects/ee-ana-zonia/assets/last_fire")
num_fires = ee.Image("projects/ee-ana-zonia/assets/num_fires")
mat_agbd = ee.Image("projects/ee-ana-zonia/assets/mature_biomass")

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si")
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec")
# mean_temp = ee.Image("projects/ee-ana-zonia/assets/mean_temp")
# yearly_anom = ee.Image("projects/ee-ana-zonia/assets/yearly_anom")

fc_sampled = age.addBands([agbd, ecoreg, indig, soil, protec, LU_sum, last_fire, num_fires, \
                    mat_agbd, yearly_si, mean_prec, age.pixelLonLat()]).updateMask(age)
#                    .stratifiedSample(10000, 'ecoreg')

task = ee.batch.Export.table.toDrive(
    collection = fc_sampled,
    description = 'tst',
    folder = 'fishnet_tiles'
)

task.start()


# fishnet = geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=0.5)
# fishnet = fishnet.map(lambda feature: feature.set('id', feature.id()))
# fishnet_info = fishnet.getInfo()['features']

# for grid_cell in fishnet_info[1:3]:
#     id = grid_cell['id']
#     grid_cell = fishnet.filter(ee.Filter.eq('id', id))
#     fc_export = fc_sampled.filterBounds(grid_cell.geometry())
    
#     task = ee.batch.Export.table.toDrive(
#         collection = fc_export,
#         description = f'{id}_tst',
#         folder = 'fishnet_tiles'
#     )
    
#     task.start()